In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.pandas.set_option('display.max_columns', None)

In [2]:
train = pd.read_csv('astro_train.csv')
test = pd.read_csv('astro_test.csv')

In [3]:
X = train.drop("class", axis=1)
y = train["class"]

In [4]:
#dataset = pd.concat([X, test])

In [5]:
#dataset.shape

In [6]:
X["err_g_log"] = np.log(X["err_g"])
X["err_i_log"] = np.log(X["err_i"])
X["err_r_cbrt"] = np.cbrt(X["err_r"])
X["err_u_log"] = np.log(X["err_u"])
X["err_z_log"] = np.log(X["err_z"])
'''dataset["#ra_cat"] = pd.cut(dataset["#ra"],
                           bins=[0,180,220,np.inf],
                           labels=[1,2,3])
dataset["field_cat"] = pd.cut(dataset["field"],
                           bins=[0,300,500,550,np.inf],
                           labels=[1,2,3,4])
dataset["extinction_r_cat"] = pd.cut(dataset["extinction_r"],
                           bins=[0,0.2,0.3,np.inf],
                           labels=[1,2,3])
dataset["dec_cat"] = pd.cut(dataset["dec"],
                           bins=[-1.25,-1.0,-0.25,np.inf],
                           labels=[1,2,3])'''

'dataset["#ra_cat"] = pd.cut(dataset["#ra"],\n                           bins=[0,180,220,np.inf],\n                           labels=[1,2,3])\ndataset["field_cat"] = pd.cut(dataset["field"],\n                           bins=[0,300,500,550,np.inf],\n                           labels=[1,2,3,4])\ndataset["extinction_r_cat"] = pd.cut(dataset["extinction_r"],\n                           bins=[0,0.2,0.3,np.inf],\n                           labels=[1,2,3])\ndataset["dec_cat"] = pd.cut(dataset["dec"],\n                           bins=[-1.25,-1.0,-0.25,np.inf],\n                           labels=[1,2,3])'

In [7]:
num_attr = ["err_z_log", "dered_z", "err_i_log", "dered_i", "photoz", "dered_r", "err_g_log", "err_r_cbrt", "dered_g", "err_u_log"]
#cat_attr = ["#ra_cat","dec_cat","extinction_r_cat","field_cat"]

In [8]:
X = X[num_attr]

In [9]:
'''num_pipeline = Pipeline([('std_scaler', StandardScaler())])
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
full_pipeline = ColumnTransformer([
               ("num", num_pipeline, num_attr),
                #("cat", OneHotEncoder(drop='first'), cat_attr),
               ])'''

'num_pipeline = Pipeline([(\'std_scaler\', StandardScaler())])\nfrom sklearn.compose import ColumnTransformer\nfrom sklearn.preprocessing import OneHotEncoder\nfull_pipeline = ColumnTransformer([\n               ("num", num_pipeline, num_attr),\n                #("cat", OneHotEncoder(drop=\'first\'), cat_attr),\n               ])'

In [10]:
#dataset = full_pipeline.fit_transform(dataset)

In [11]:
#dataset.shape

In [12]:
#train_prepared = dataset[:45000,:]
#test_prepared = dataset[45000:, :]

In [13]:
#train_prepared.shape

In [14]:
#test_prepared.shape
seed = 1234
num_folds = 10
#n_jobs = -1

In [15]:
#over-sampling data
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_res, y_res = smote.fit_sample(X, y)

Using TensorFlow backend.


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=seed)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, KFold

models = [('rfc', RandomForestClassifier(n_estimators=600, class_weight={0:0.7, 1:1.11, 2:1.9}))]
additions = [('PCA', PCA(n_components=4))]
scalers = [('StandardScaler', StandardScaler()),
           ('MinMaxScaler', MinMaxScaler()),
           ('MaxAbsScaler', MaxAbsScaler()),
           ('RobustScaler', RobustScaler()),
           ('QuantileTransformer-Normal', QuantileTransformer(output_distribution='normal')),
           ('QuantileTransformer-Uniform', QuantileTransformer(output_distribution='uniform')),
           ('PowerTransformer-Yeo-Johnson', PowerTransformer(method='yeo-johnson')),
           ('Normalizer', Normalizer())
          ]

In [18]:
pipelines = []

for model in models:
    # Append only model
    model_name = "_" + model[0]
    pipelines.append((model_name, Pipeline([model])))
    
    for addition in additions:
        model_name = "_" + model[0] + "-" + addition[0]
        pipelines.append((model_name, Pipeline([addition, model])))
    
    # Append model+scaler
    for scalar in scalers:
        model_name = scalar[0] + "_" + model[0]
        pipelines.append((model_name, Pipeline([scalar, model])))
        
print("Created these pipelines:")
for pipe in pipelines:
    print(pipe[0])

Created these pipelines:
_rfc
_rfc-PCA
StandardScaler_rfc
MinMaxScaler_rfc
MaxAbsScaler_rfc
RobustScaler_rfc
QuantileTransformer-Normal_rfc
QuantileTransformer-Uniform_rfc
PowerTransformer-Yeo-Johnson_rfc
Normalizer_rfc


In [ ]:
results = []
names = []
test_scores = []
avg_f1_score = []

for name, model in pipelines:
        kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
        cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring="accuracy")
        results.append(cv_results)
        names.append(name)

        # Print CV results of the best CV classier
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
    
        # fit on train and predict on test
        model.fit(X_train, y_train)
        
        curr_test_score = model.score(X_test, y_test)
        
        test_scores.append(curr_test_score)
        
        y_pred = model.predict(X_test)
        avg_f1_score.append(sum(f1_score(y_test, y_pred, average=None))/3)

_rfc: 0.737980 (0.007828)
_rfc-PCA: 0.664877 (0.006122)
StandardScaler_rfc: 0.737382 (0.008940)
MinMaxScaler_rfc: 0.737959 (0.007480)
MaxAbsScaler_rfc: 0.738194 (0.007910)
RobustScaler_rfc: 0.738579 (0.006818)
QuantileTransformer-Normal_rfc: 0.737510 (0.007512)
QuantileTransformer-Uniform_rfc: 0.738900 (0.008148)


In [0]:
result = pd.DataFrame({'Model':names, 'Train score':[acc.mean() for acc in results], 'Test score':test_scores, 'Avg F1 score':avg_f1_score})

In [0]:
'''scores = cross_val_score(rfc, X_train, y_train, cv=3, scoring='accuracy')
scores.mean()
rfc.fit(X_train, y_train)
ypred = rfc.predict(X_test)
from sklearn.metrics import confusion_matrix, classification_report
confusion_matrix(y_test, ypred)
print(classification_report(y_test, ypred))
from collections import Counter
Counter(ypred)
final_pred = rfc.predict(test_prepared)
Counter(final_pred)'''

In [23]:
result

,Model,Train score,Test score,Avg F1 score
0,_rfc,0.740632,0.740462,0.740073
1,_rfc-PCA,0.663466,0.669718,0.668563
2,StandardScaler_rfc,0.740461,0.741488,0.741044
3,MinMaxScaler_rfc,0.740846,0.741146,0.740783
4,MaxAbsScaler_rfc,0.740419,0.739264,0.738871
5,RobustScaler_rfc,0.739734,0.738837,0.738455
6,QuantileTransformer-Normal_rfc,0.740354,0.742173,0.741891
7,QuantileTransformer-Uniform_rfc,0.741317,0.739778,0.739395
8,PowerTransformer-Yeo-Johnson_rfc,0.739841,0.740975,0.740535
9,Normalizer_rfc,0.733831,0.737639,0.737437


In [0]:
final_model = Pipeline([('QuantileTransformer-Normal', QuantileTransformer(output_distribution='normal')),
                        ('rfc', RandomForestClassifier(n_estimators=600, class_weight={0:0.7, 1:1.11, 2:1.9}))
                      ])

In [28]:
final_model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('QuantileTransformer-Normal',
                 QuantileTransformer(copy=True, ignore_implicit_zeros=False,
                                     n_quantiles=1000,
                                     output_distribution='normal',
                                     random_state=None, subsample=100000)),
                ('rfc',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight={0: 0.7, 1: 1.11, 2: 1.9},
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fra

In [0]:
test["err_g_log"] = np.log(test["err_g"])
test["err_i_log"] = np.log(test["err_i"])
test["err_r_cbrt"] = np.cbrt(test["err_r"])
test["err_u_log"] = np.log(test["err_u"])
test["err_z_log"] = np.log(test["err_z"])

final_test = test[num_attr]

In [0]:
y_pred = final_model.predict(final_test)

In [33]:
from collections import Counter
Counter(y_pred)

Counter({0: 6105, 1: 5269, 2: 3626})

In [0]:
sub = pd.DataFrame({"id":test["id"], "class":y_pred})

In [0]:
sub.to_csv("submission.csv",index=False)

In [0]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [0]:
models = [LinearDiscriminantAnalysis(), SVC(), KNeighborsClassifier()]

In [39]:
for model in models:
  kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring="accuracy")
  # Print CV results of the best CV classier
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

Normalizer_rfc: 0.594512 (0.005284)
Normalizer_rfc: 0.611772 (0.005435)
Normalizer_rfc: 0.682736 (0.006822)
